In [7]:
import yt
import glob
from unyt import unyt_array
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

cent = 2.5e8

files = glob.glob ("def_high_dens_hdf5_chk_000021")
files.sort()

tag_file = pd.read_csv("p_tag_all.dat", sep = "\s+", usecols = [0, 1, 4], names = ["p_tag", "Fe", "Cr"])

tags = tag_file["p_tag"].to_numpy()

def iron_clumps(pfilter,data):
    p_tags = data[(pfilter.filtered_type, "particle_tag")]
    clumps = np.in1d(p_tags, tags)
    return yt.YTArray(clumps)
    
yt.add_particle_filter("iron_clumps", function = iron_clumps, filtered_type = "all", requires = ["particle_tag"])

for file in files:
    ds = yt.load(file)
    def binding_energy(field, data):
        return(data["gas", "density"] * 
               ((0.5 * ((data[("flash", "velx")])**2 + (data[("flash", "vely")])**2)) + 
                data[("flash", "eint")] + data[("flash", "gpot")]))

    ds.add_field(("flash", "bneg"), 
                 function = binding_energy, 
                 sampling_type = "cell", 
                 units = "erg/g")
    ds.add_particle_filter("iron_clumps")
    
    def X_iron(field,data):
        A = data[("iron_clumps", "particle_tag")]
        A = A.tolist()
        filtered_p_tag = tag_file.loc[tag_file['p_tag'].isin(A)]
        filtered_p_tag = filtered_p_tag.reset_index(drop = True)
        filtered_p_tag = filtered_p_tag.iloc[[i for i, x in sorted(enumerate(filtered_p_tag['p_tag']), key=lambda x: A.index(x[1]))], :]
        arr_x = filtered_p_tag["Fe"].to_numpy()
        X_fe = unyt_array(arr_x, "dimensionless")
        return (X_fe)
    
    ds.add_field(("iron_clumps", "X_fe"), function = X_iron, sampling_type = "particle", units = 'dimensionless')
    

    
    ad = ds.all_data()
    box = ds.region(center = [cent,0,0], left_edge = [0,-2*cent,0], right_edge = [2*cent,2*cent,2*cent])
    
    for x in ds.derived_field_list:
        print(x)
    #print(len(ad['deposit', 'iron_clumps_nn_X_fe']))
    #print(ad["unbound_particles", "X_fe"])
    
    positive_binding_energy = ad.cut_region(['obj["flash", "bneg"] > 1.0e17'])
    negative_binding_energy = ad.cut_region(['obj["flash", "bneg"] < -1.0e17'])
    pos_bn_eg_without_fluff = positive_binding_energy.cut_region(['obj["gas", "density"] > 1.0e-2'])
    neg_bn_eg_without_fluff = negative_binding_energy.cut_region(['obj["gas", "density"] > 1.0e-2'])
    #bneg_new = ad.exclude_inside(("flash", "bneg"), -1.0e17, 1.0e17)
    #bn_eg_without_fluff = bneg_new.exclude_below(("gas", "density"), 1.0e-2)
    
    #print(len(ds.r[('all', 'cell_gas_temperature')]))
#    print((ad[('index', 'grid_indices')]))
    
    #slc3 = yt.SlicePlot(ds, 'theta', ("flash", "dens"), center=[cent,0,0])
    #slc3.set_zlim(("flash","bneg"), -1.0e24, 1.0e22)
    #slc3.set_log(("flash","bneg"), linthresh = 1.0e20)
    #slc3.set_cmap (field = ("flash", "bneg"), cmap = "inferno")
    #slc3.annotate_title("Specific Binding Energy Density")#
    #slc3.annotate_timestamp (corner='upper_right', draw_inset_box=True, redshift = False)
    #slc3.annotate_contour(('deposit', 'iron_clumps_nn_X_fe'))
#    slc3.annotate_contour(("flash", "Cr_dens"),levels = 1,take_log = False)
    #slc3.set_width((2*cent, 4*cent))
    #slc3.show()
    #slc3.save("contour_plot_1_.png")

yt : [WARNING  ] 2023-03-17 19:53:50,666 The iron_clumps particle filter already exists. Overriding.
yt : [INFO     ] 2023-03-17 19:53:50,672 Particle file found: def_high_dens_hdf5_chk_000021
yt : [WARNING  ] 2023-03-17 19:53:50,685 Extending theta dimension to 2PI + left edge.
yt : [INFO     ] 2023-03-17 19:53:50,693 Parameters: current_time              = 2.100037265112491
yt : [INFO     ] 2023-03-17 19:53:50,694 Parameters: domain_dimensions         = [ 64 128   1]
yt : [INFO     ] 2023-03-17 19:53:50,694 Parameters: domain_left_edge          = [ 0.00000e+00 -1.31072e+10  0.00000e+00]
yt : [INFO     ] 2023-03-17 19:53:50,695 Parameters: domain_right_edge         = [1.31072000e+10 1.31072000e+10 6.28318531e+00]
yt : [INFO     ] 2023-03-17 19:53:50,695 Parameters: cosmological_simulation   = 0.0
yt : [WARNING  ] 2023-03-17 19:53:51,317 In cylindrical geometry, gradient fields may contain artifacts near cartesian axes.
yt : [WARNING  ] 2023-03-17 19:53:51,317 In cylindrical geometry, 

AttributeError: 'FLASHDataset' object has no attribute 'add_nearest_neighbor_field'